### 1. Importing Pandas, the only library needed here.

In [3]:
import pandas as pd

### 2. Loading the datasets for the FIPS codes the Air Quality Index data for each county.

FIPS (Federal Information Processing Standards) codes are a system designed by the main standards agency of the United States Government, the National Institute of Standards and Technology (NIST). Each location in the United States, whether it is a state, county, city, metro area is assigned a specific code. These codes are especially useful when we need to combine different location-specific data. County FIPS codes are 5 digits. Only the AQI data doesn't have them in this project so we'll need to add it by hand.  

In [4]:
# Loading FIPS data
fips_df = pd.read_csv("FIPS_DATA_State__County_and_City_FIPS_Reference_Table_20250413.csv", dtype={"StCnty FIPS Code": str})[:-2]

# Loading AQI (target) data
aqi_df = pd.read_csv("AIR_annual_aqi_by_county_2019.csv")

aqi_df.head()

,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10
0,Alabama,Baldwin,2019,271,221,50,0,0,0,0,80,55,40,0,0,198,73,0
1,Alabama,Clay,2019,107,75,32,0,0,0,0,71,56,41,0,0,0,107,0
2,Alabama,Colbert,2019,263,238,25,0,0,0,0,65,50,38,0,0,219,44,0
3,Alabama,DeKalb,2019,361,303,58,0,0,0,0,90,54,39,0,0,306,55,0
4,Alabama,Elmore,2019,228,208,20,0,0,0,0,100,50,39,0,0,228,0,0


### 3. Reducing the FIPS data to only the variables necessary and dropping duplicates
Since the FIPS dataset has a row entry for each settlement inside a county which is not necessary for this project. In the end we'll have a unique row for each county instead.

In [5]:
# Reduce FIPS data to only necessary variables
fips_subset = fips_df[["State Name", "County Name", "StCnty FIPS Code"]]

# Drop duplicates
fips_dedup = fips_subset.drop_duplicates(subset=["State Name", "County Name"])

fips_dedup.head()

,State Name,County Name,StCnty FIPS Code
0,ALABAMA,AUTAUGA,01001
6,ALABAMA,BALDWIN,01003
30,ALABAMA,BARBOUR,01005
38,ALABAMA,BIBB,01007
47,ALABAMA,BLOUNT,01009


### 4. "Cleaning" names
We strip and convert all characters to uppercase in the state and county names for both datasets so that we can match them easily afterward.

In [6]:
# Turning everything to same character format
clean_name = lambda x : x.strip().upper()

# Apply the cleaning function to Dataset 1 (FIPS data)
fips_dedup['State_clean'] = fips_dedup['State Name'].apply(clean_name)
fips_dedup['County_clean'] = fips_dedup['County Name'].apply(clean_name)

# Apply the cleaning function to Dataset 2 (Target data)
aqi_df['State_clean'] = aqi_df['State'].apply(clean_name)
aqi_df['County_clean'] = aqi_df['County'].apply(clean_name)

C:\Users\Ali\AppData\Local\Temp\ipykernel_17860\3215889540.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_dedup['State_clean'] = fips_dedup['State Name'].apply(clean_name)
C:\Users\Ali\AppData\Local\Temp\ipykernel_17860\3215889540.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_dedup['County_clean'] = fips_dedup['County Name'].apply(clean_name)


### 5. Merging datasets on state and county names.
We merge the datasets on both state and county names since there are multiple counties with the same name across different states, though there aren't any same-named counties within a single state.

Most of them are named after former presidents: 24 counties are called "Lincoln", 25 counties are called "Franklin", 26 counties are called "Jefferson" and 31 counties are called "Washington".

In [7]:
# Merge on both cleaned state and county names
merged_df = pd.merge(aqi_df, 
                     fips_dedup, 
                     left_on=['State_clean', 'County_clean'], 
                     right_on=['State_clean', 'County_clean'], 
                     how='left')

### 6. Cleaning the final dataframe

The resulting merged dataframe has its unnecessary columns dropped, its column order reorganized for better readability and the FIPS code is converted to a string type because some FIPS codes have a leading zero which Python can't handle in integer form. Finally, the dataframe is converted to a csv file.

In [8]:
# Drop unnecessary columns
merged_df = merged_df.drop(columns=[
    'State_clean', 'County_clean', 'State Name', 'County Name'
])

# Move 'StCnty FIPS Code' between 'County' and 'Year'
cols = merged_df.columns.tolist()
cols.remove('StCnty FIPS Code')
insert_position = cols.index('Year')  # Insert before 'Year'
cols.insert(insert_position, 'StCnty FIPS Code')

# Reorder DataFrame
merged_df = merged_df[cols]

merged_df['StCnty FIPS Code'] = merged_df['StCnty FIPS Code'].astype(str)

# Check the result to ensure the merge worked as expected
merged_df.to_csv('AIR_DATA_WITH_FIPS.csv', index=False)